In [2]:
from sagemaker import get_execution_role #this contains the permissions to access the bucket
import boto3 #aws python client
import pandas as pd
import io #used for conversion to csv
s3 = boto3.client('s3') #connect to S3
bucketName = "cs539-e2023-149b5d57-81a4-4e31-a476-c81e7e07dc33"

In [ ]:
allnames = ['business', 'checkin', 'review', 'tip', 'user']
names = ['review', 'tip', 'user']
for k in names:
    key = 'yelp_academic_dataset_' + k
    jsonkey = 'datasets/json/' + key + '.json'
    csvkey = 'datasets/csv/' + key + '.csv'
    tsvkey = 'datasets/tsv/' + key + '.tsv'
    print("Reading from ", jsonkey)
    response = s3.get_object(Bucket=bucketName, Key=jsonkey)
    print("Parsing Data")
    df = pd.read_json(response.get("Body"), lines=True)
    print("Writing to ", tsvkey)
    with io.StringIO() as tsv_buffer: #buffer to write the data
        df.to_csv(tsv_buffer, sep="\t", index=False)
        s3.put_object(Bucket=bucketName, Key=tsvkey, Body=tsv_buffer.getvalue())

Reading from  datasets/json/yelp_academic_dataset_review.json
Parsing Data


In [9]:
k = 'user'
key = 'yelp_academic_dataset_' + k
jsonkey = 'datasets/json/' + key + '.json'
csvkey = 'datasets/csv/' + key + '.csv'
tsvkey = 'datasets/tsv/' + key + '.tsv'
print("Reading from ", jsonkey)
response = s3.get_object(Bucket=bucketName, Key=jsonkey)
print("Parsing Data")
df = pd.read_json(response.get("Body"), lines=True)

Reading from  datasets/json/yelp_academic_dataset_user.json
Parsing Data


In [10]:
print("Writing to ", csvkey)
with io.StringIO() as csv_buffer: #buffer to write the data
    df.to_csv(csv_buffer, index=False)
    s3.put_object(Bucket=bucketName, Key=csvkey, Body=csv_buffer.getvalue())

Writing to  datasets/csv/yelp_academic_dataset_user.csv


In [11]:
print("Writing to ", tsvkey)
with io.StringIO() as tsv_buffer: #buffer to write the data
    df.to_csv(tsv_buffer, sep="\t", index=False)
    s3.put_object(Bucket=bucketName, Key=tsvkey, Body=tsv_buffer.getvalue())

Writing to  datasets/tsv/yelp_academic_dataset_user.tsv


In [3]:
key = 'yelp_academic_dataset_' + 'review'
jsonkey = 'datasets/json/' + key + '.json'
print("Reading from ", jsonkey)
response = s3.get_object(Bucket=bucketName, Key=jsonkey)
print("Parsing Data")
df = pd.read_json(response.get("Body"), lines=True)

Reading from  datasets/json/yelp_academic_dataset_review.json
Parsing Data


In [9]:
import re
REGEX_WHITESPACE = re.compile(r"\s+")
REGEX_INVALID_CHARS = re.compile(r'[,\-]')

def cleanText(text):
    #remove newlines and tabs and spaces
    text = REGEX_WHITESPACE.sub(' ', text)
    text = REGEX_INVALID_CHARS.sub('',text)
    return text
    
def getType(val):
    return type(val).__name__

In [6]:
df['type'] = df['text'].map(getType)
df

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,type
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11,str
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18,str
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30,str
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03,str
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15,str
...,...,...,...,...,...,...,...,...,...,...
6990275,H0RIamZu0B0Ei0P4aeh3sQ,qskILQ3k0I_qcCMI-k6_QQ,jals67o91gcrD4DC81Vk6w,5,1,2,1,Latest addition to services from ICCU is Apple...,2014-12-17 21:45:20,str
6990276,shTPgbgdwTHSuU67mGCmZQ,Zo0th2m8Ez4gLSbHftiQvg,2vLksaMmSEcGbjI5gywpZA,5,2,1,2,"This spot offers a great, affordable east week...",2021-03-31 16:55:10,str
6990277,YNfNhgZlaaCO5Q_YJR4rEw,mm6E4FbCMwJmb7kPDZ5v2Q,R1khUUxidqfaJmcpmGd4aw,4,1,0,0,This Home Depot won me over when I needed to g...,2019-12-30 03:56:30,str
6990278,i-I4ZOhoX70Nw5H0FwrQUA,YwAMC-jvZ1fvEUum6QkEkw,Rr9kKArrMhSLVE9a53q-aA,5,1,0,0,For when I'm feeling like ignoring my calorie-...,2022-01-19 18:59:27,str


In [8]:
df.loc[df['type'] != 'str']

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,type


In [10]:
df['text'] = df['text'].map(cleanText)
df

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,type
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,0,0,0,If you decide to eat here just be aware it is ...,2018-07-07 22:09:11,str
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18,str
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30,str
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5,1,0,1,Wow! Yummy different delicious. Our favorite i...,2015-01-04 00:01:03,str
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15,str
...,...,...,...,...,...,...,...,...,...,...
6990275,H0RIamZu0B0Ei0P4aeh3sQ,qskILQ3k0I_qcCMI-k6_QQ,jals67o91gcrD4DC81Vk6w,5,1,2,1,Latest addition to services from ICCU is Apple...,2014-12-17 21:45:20,str
6990276,shTPgbgdwTHSuU67mGCmZQ,Zo0th2m8Ez4gLSbHftiQvg,2vLksaMmSEcGbjI5gywpZA,5,2,1,2,This spot offers a great affordable east weeke...,2021-03-31 16:55:10,str
6990277,YNfNhgZlaaCO5Q_YJR4rEw,mm6E4FbCMwJmb7kPDZ5v2Q,R1khUUxidqfaJmcpmGd4aw,4,1,0,0,This Home Depot won me over when I needed to g...,2019-12-30 03:56:30,str
6990278,i-I4ZOhoX70Nw5H0FwrQUA,YwAMC-jvZ1fvEUum6QkEkw,Rr9kKArrMhSLVE9a53q-aA,5,1,0,0,For when I'm feeling like ignoring my caloriec...,2022-01-19 18:59:27,str


In [14]:
df = df.drop(columns=['type'])
df

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,0,0,0,If you decide to eat here just be aware it is ...,2018-07-07 22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5,1,0,1,Wow! Yummy different delicious. Our favorite i...,2015-01-04 00:01:03
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15
...,...,...,...,...,...,...,...,...,...
6990275,H0RIamZu0B0Ei0P4aeh3sQ,qskILQ3k0I_qcCMI-k6_QQ,jals67o91gcrD4DC81Vk6w,5,1,2,1,Latest addition to services from ICCU is Apple...,2014-12-17 21:45:20
6990276,shTPgbgdwTHSuU67mGCmZQ,Zo0th2m8Ez4gLSbHftiQvg,2vLksaMmSEcGbjI5gywpZA,5,2,1,2,This spot offers a great affordable east weeke...,2021-03-31 16:55:10
6990277,YNfNhgZlaaCO5Q_YJR4rEw,mm6E4FbCMwJmb7kPDZ5v2Q,R1khUUxidqfaJmcpmGd4aw,4,1,0,0,This Home Depot won me over when I needed to g...,2019-12-30 03:56:30
6990278,i-I4ZOhoX70Nw5H0FwrQUA,YwAMC-jvZ1fvEUum6QkEkw,Rr9kKArrMhSLVE9a53q-aA,5,1,0,0,For when I'm feeling like ignoring my caloriec...,2022-01-19 18:59:27


In [15]:
trimmed_df = df[['review_id', 'text']]
trimmed_df

,review_id,text
0,KU_O5udG6zpxOg-VcAEodg,If you decide to eat here just be aware it is ...
1,BiTunyQ73aT9WBnpR9DZGw,I've taken a lot of spin classes over the year...
2,saUsX_uimxRlCVr67Z4Jig,Family diner. Had the buffet. Eclectic assortm...
3,AqPFMleE6RsU23_auESxiA,Wow! Yummy different delicious. Our favorite i...
4,Sx8TMOWLNuJBWer-0pcmoA,Cute interior and owner (?) gave us tour of up...
...,...,...
6990275,H0RIamZu0B0Ei0P4aeh3sQ,Latest addition to services from ICCU is Apple...
6990276,shTPgbgdwTHSuU67mGCmZQ,This spot offers a great affordable east weeke...
6990277,YNfNhgZlaaCO5Q_YJR4rEw,This Home Depot won me over when I needed to g...
6990278,i-I4ZOhoX70Nw5H0FwrQUA,For when I'm feeling like ignoring my caloriec...


In [16]:
tsvFullkey = 'datasets/tsv/' + key + '.tsv'
print("Writing full to ", tsvFullkey)
with io.StringIO() as tsv_buffer: #buffer to write the data
    df.to_csv(tsv_buffer, sep="\t", index=False)
    s3.put_object(Bucket=bucketName, Key=tsvFullkey, Body=tsv_buffer.getvalue())
tsvTrimkey = 'datasets/tsv/' + key + '_trimmed.tsv'
print("Writing full to ", tsvTrimkey)
with io.StringIO() as tsv_buffer: #buffer to write the data
    trimmed_df.to_csv(tsv_buffer, sep="\t", index=False)
    s3.put_object(Bucket=bucketName, Key=tsvTrimkey, Body=tsv_buffer.getvalue())

Writing full to  datasets/tsv/yelp_academic_dataset_review.tsv
Writing full to  datasets/tsv/yelp_academic_dataset_review_trimmed.tsv
